<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Subsampling-tree" data-toc-modified-id="Subsampling-tree-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Subsampling tree</a></span></li><li><span><a href="#Creating-datasets" data-toc-modified-id="Creating-datasets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Creating datasets</a></span></li><li><span><a href="#parafit:-just-global-test" data-toc-modified-id="parafit:-just-global-test-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>parafit: just global test</a></span><ul class="toc-item"><li><span><a href="#Saving-results" data-toc-modified-id="Saving-results-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Saving results</a></span></li><li><span><a href="#Summarizing" data-toc-modified-id="Summarizing-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Summarizing</a></span></li></ul></li><li><span><a href="#--TO-HERE--" data-toc-modified-id="--TO-HERE---7"><span class="toc-item-num">7&nbsp;&nbsp;</span>--TO HERE--</a></span></li><li><span><a href="#parafit:-individual-links" data-toc-modified-id="parafit:-individual-links-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>parafit: individual links</a></span><ul class="toc-item"><li><span><a href="#Saving-results" data-toc-modified-id="Saving-results-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Saving results</a></span></li><li><span><a href="#Summarizing" data-toc-modified-id="Summarizing-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Summarizing</a></span></li><li><span><a href="#Links-per-host" data-toc-modified-id="Links-per-host-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Links per host</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>sessionInfo</a></span></li><li><span><a href="#Sandbox" data-toc-modified-id="Sandbox-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Sandbox</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#notes" data-toc-modified-id="notes-10.0.0.1"><span class="toc-item-num">10.0.0.1&nbsp;&nbsp;</span>notes</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Goal

* Run `ape::parafit()` analysis on IndD dataset summed by host
* Sensitivity analysis (species-level): selecting one sample per species

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/host_specific/parafit/'

# phyloseq
physeq_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/phyloseq/physeq_r5k.RDS'

# host phylogeny
host_tree_file = '/ebio/abt3_projects/Georg_animal_feces/data/animal/phylogeny/TT/unified_metadata_complete_20180625_rn_16S-r5k_IndD.nwk'

# misc
sparsity_cutoff = 0.05
threads = 12

# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(phyloseq)
library(ape)

library(future)
library(future.batchtools)
library(future.apply)

source('/ebio/abt3_projects/Georg_animal_feces/code/init.R') 
source('/ebio/abt3_projects/Georg_animal_feces/code/itol.R') 


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘future.apply’

The following object is masked from ‘package:future’:

    future_lapply



In [3]:
make_dir(work_dir)
setwd(work_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/host_specific/parafit/ 


In [4]:
rescale_dist_mtx = function(m){
    m = m %>% as.matrix
    labs = m %>% colnames
    n_row = m %>% nrow
    n_col = m %>% ncol
    x = m %>% as.vector 
    x = scales::rescale(x) 
    m = matrix(x, nrow=n_row, ncol=n_col)
    colnames(m) = labs
    rownames(m) = labs
    m = m %>% as.dist
    return(m)
}

# Load

In [5]:
physeq = readRDS(physeq_file) %>%
    filter_taxa(function(x) sum(x > 0) / length(x) > sparsity_cutoff, TRUE) %>%
    subset_samples(class == 'Mammalia')
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 408 taxa and 160 samples ]
sample_data() Sample Data:       [ 160 samples by 171 sample variables ]
tax_table()   Taxonomy Table:    [ 408 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 408 tips and 406 internal nodes ]

In [6]:
metadata = physeq %>%
    sample_data %>%
    dplyr::select(scientific_name) %>%
    mutate(SampleID = rownames(.)) %>%
    as.data.frame

rownames(metadata) = metadata$SampleID
metadata %>% dfhead

Warning message in class(x) <- c(subclass, "tbl_df", "tbl", "data.frame"):
“Setting class(x) to multiple strings ("tbl_df", "tbl", ...); result will no longer be an S4 object”

[1] 160   2


,scientific_name,SampleID
X1.Eurasian.Beaver,Castor_fiber,X1.Eurasian.Beaver
X101.Horse,Equus_caballus,X101.Horse
X102.Kulan,Equus_hemionus,X102.Kulan


In [7]:
host_tree = read.tree(host_tree_file)
host_tree


Phylogenetic tree with 213 tips and 177 internal nodes.

Tip labels:
	X256.Zander, F302.Northern.Pike, X188.Brook.Trout, F300.Asp, X330.Ide, X327.Silver.Bream, ...
Node labels:
	Node1, Node2, Node3, Node4, Node5, Node6, ...

Rooted; includes branch lengths.

# Subsampling tree

In [8]:
# number of subsampled trees
ntrees = 100

In [9]:
#' randomly selecting one per group
tree_subsample = function(L, df, tree){
    # get subsample (note: subsampling within each species)
    to_keep = df %>% 
        group_by(scientific_name) %>% 
        sample_n(1) %>%
        .$sample
    # subsampling tree
    to_rm = setdiff(tree$tip.label, to_keep)
    tree = drop.tip(tree, to_rm)
    return(tree)
}

In [10]:
# subsampling trees
df = metadata %>%
    mutate(sample = rownames(.)) %>%
    dplyr::select(sample, scientific_name) 

doParallel::registerDoParallel(threads)
host_tree_l = plyr::llply(as.list(1:ntrees), 
                          function(x) tree_subsample(x, df, host_tree),
                          .parallel=TRUE)

# tree lengths
host_tree_l %>% length %>% print
lapply(host_tree_l, function(x) x$tip.label %>% length) %>% unlist %>% summary

[1] 100


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
     82      82      82      82      82      82 

In [11]:
# checking overlap of labels
setdiff(host_tree_l[[1]]$tip.label, rownames(metadata)) %>% length %>% print

[1] 0


# Creating datasets

In [12]:
make_parafit_input = function(host_tree, physeq){
    # subsampled phyloseq object
    physeq_f = prune_samples(sample_names(physeq) %in% host_tree$tip.label, 
                             physeq) 

    # microbial tree distance matrix
    micro_D = physeq_f %>%
        phy_tree %>% cophenetic %>% 
        rescale_dist_mtx %>% as.matrix
    # host tree distance matrix
    host_D = host_tree %>% cophenetic %>% 
             rescale_dist_mtx %>% as.matrix
    
    # otu pres/abs matrix
    otu = physeq_f %>% 
        otu_table %>%
        as.data.frame %>%
        as.matrix %>%
        t %>%
        apply(2, function(x) ifelse(x > 0, 1, 0)) %>% 
        as.matrix                
              
              
    # checking overlap
    x = length(intersect(rownames(otu), rownames(host_D)))
    y = length(union(rownames(otu), rownames(host_D)))
    stopifnot(x == y)
              
    # preparing paco data
    return(list(host.D=host_D, para.D = micro_D, HP=otu))
}

In [13]:
# creating list of parafit input objects
doParallel::registerDoParallel(threads)
PF_l = plyr::llply(host_tree_l, make_parafit_input, physeq=physeq, .parallel=TRUE)
PF_l %>% length

[1] 100

# parafit: just global test

In [14]:
# cluster resources
resources = list(h_rt = '00:59:00',
                 h_vmem = '10G',
                 threads = '1',
                 conda.env = 'py3_physeq_eco', 
                 conda.path = "/ebio/abt3_projects/software/miniconda3_gt4.4/bin")
plan(batchtools_sge, resources=resources)

In [15]:
# Running parafit on each subsample
## WARNING: SLOW!
parafit_each = function(L, nperm=99, seed=3843, correction="cailliez", test.links=TRUE){
    parafit(L$host.D, L$para.D, L$HP, 
            nperm=nperm, 
            seed=seed,
            correction=correction,
            test.links=test.links)
}

PF_res_l = future_lapply(PF_l, parafit_each, 
                         nperm=999, 
                         test.links=FALSE,
                         future.packages=c('ape'))

Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vec

“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'NULL'”

In [16]:
# results
PF_res_l %>% length

[1] 100

## Saving results

In [17]:
# for loading results instead of re-running
PF_file = file.path(work_dir, 'physeq_IndD-sens-justMam_parafit.RDS')
PF_file

[1] "/ebio/abt3_projects/Georg_animal_feces/data/host_specific/parafit//physeq_IndD-sens-justMam_parafit.RDS"

In [19]:
# saving
saveRDS(PF_res_l, file=PF_file)
cat('File written:', PF_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/host_specific/parafit//physeq_IndD-sens-justMam_parafit.RDS 


In [20]:
# re-loading
PF_res_l = readRDS(PF_file)
PF_res_l %>% length

[1] 100

In [ ]:
send_email('http://127.0.0.1:9705/notebooks/notebooks/16S/4_host_specific/parafit/01b_IndD-sens-justMammalia.ipynb')

## Summarizing

In [23]:
# distribution of effect size
lapply(PF_res_l, function(x) x$ParaFitGlobal) %>% unlist %>% summary %>% print

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  305.7   403.9   452.7   444.2   491.5   553.2 


In [24]:
# distribution of p-values
lapply(PF_res_l, function(x) x$p.global) %>% unlist %>% summary %>% print

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.001   0.001   0.001   0.001   0.001   0.001 


# --TO HERE--

# parafit: individual links

In [ ]:
# cluster resources
resources = list(h_rt = '200:00:00',
                 h_vmem = '10G',
                 threads = '1',
                 conda.env = 'py3_physeq_eco', 
                 conda.path = "/ebio/abt3_projects/software/miniconda3_gt4.4/bin")
plan(batchtools_sge, resources=resources)

In [ ]:
# Running parafit on each subsample
## WARNING: SLOW!
parafit_each = function(L, nperm=99, seed=3843, correction="cailliez", test.links=TRUE){
    parafit(L$host.D, L$para.D, L$HP, 
            nperm=nperm, 
            seed=seed,
            correction=correction,
            test.links=test.links)
}

PF_res_links_l = future_lapply(PF_l, parafit_each, 
                         nperm=999, 
                         test.links=TRUE,
                         future.packages=c('ape'))

In [ ]:
# results
PF_res_links_l %>% length

## Saving results

In [ ]:
# for loading results instead of re-running
PF_file = file.path(work_dir, 'physeq_IndD-sens_parafit-links.RDS')
PF_file

In [ ]:
# saving
saveRDS(PF_res_links_l, file=PF_file)
cat('File written:', PF_file, '\n')

In [ ]:
# re-loading
PF_res_links_l = readRDS(PF_file)
PF_res_links_l %>% length

In [ ]:
send_email('http://127.0.0.1:9705/notebooks/notebooks/16S/4_host_specific/parafit/01_IndD-sens.ipynb')

## Summarizing

In [ ]:
# distribution of effect size
lapply(PF_res_links_l, function(x) x$ParaFitGlobal) %>% unlist %>% summary

In [ ]:
# distribution of p-values
lapply(PF_res_links_l, function(x) x$p.global) %>% unlist %>% summary

## Links per host

* distribution of links per host

In [ ]:
# format
para_per_host = lapply(PF_res_links_l, function(x) x$para.per.host) %>%
    do.call(rbind, .) %>%
    as.data.frame %>%
    mutate(Dataset_rep = 1:nrow(.)) %>%
    gather(Sample, N_links, -Dataset_rep) 
               
para_per_host %>% dfhead

In [ ]:
# adding metadata
para_per_host = physeq %>%
    sample_data %>%
    as.matrix %>%
    as.data.frame %>%
    dplyr::select(X.SampleID, class, order, family, genus, scientific_name) %>%
    inner_join(para_per_host, c('X.SampleID'='Sample')) 

para_per_host %>% dfhead

In [ ]:
# summarizing by species: check
para_per_host %>%
    group_by(scientific_name) %>%
    summarize(n=n()) %>%
    ungroup() %>%
    .$n %>% summary %>% print

In [ ]:
# summarizing by species
alpha=0.95

para_per_host = para_per_host %>%
    group_by(scientific_name, class, order, family, genus) %>%
    summarize(min_n_links = min(N_links),
              q1_n_links = quantile(N_links, 0.25),
              mean_n_links = mean(N_links),
              median_n_links = median(N_links),
              q3_n_links = quantile(N_links, 0.75),
              max_n_links = max(N_links),
              sd_n_links = sd(N_links),
              CI_low = quantile(N_links, (1-alpha)/2),
              CI_high = quantile(N_links, alpha+((1-alpha)/2))) %>%
    ungroup() %>%
    mutate(scientific_name = scientific_name %>% reorder(-median_n_links))

para_per_host$CI_low %>% summary %>% print
para_per_host %>% dfhead

In [ ]:
# plotting distribution
p = ggplot(para_per_host, aes(scientific_name, median_n_links, color=class)) +
    geom_linerange(aes(ymin=median_n_links-sd_n_links,
                       ymax=median_n_links+sd_n_links)) +
    geom_point() +
    theme_bw() +
    theme(
        axis.text.x = element_blank()
    )

options(repr.plot.width=10, repr.plot.height=4)
plot(p)

In [ ]:
# plotting distribution
p = para_per_host %>%
    ggplot(aes(class, mean_n_links)) +
    geom_boxplot() +
    scale_y_log10() +
    labs(x='Host class', y='No. of host-symbiont links') +
    theme_bw() +
    theme(
        axis.text.x = element_text(angle=45, hjust=1)
    )

options(repr.plot.width=4, repr.plot.height=3)
plot_write(p, file='IndD-sens_parafit_nLinks-byClass.pdf')

In [ ]:
# # plotting distribution
# p = para_per_host %>%
#     filter(CI_low > 0,
#            class == 'Mammalia') %>%
#     mutate(scientific_name = scientific_name %>% reorder(median_n_links)) %>%
#     ggplot(aes(scientific_name, median_n_links, color=class)) +
#     geom_linerange(aes(ymin=median_n_links-sd_n_links,
#                        ymax=median_n_links+sd_n_links)) +
#     geom_point() +
#     labs(y='No. of links (95% CI)', x='Host') +
#     coord_flip() +
#     theme_bw() +
#     theme(
#         axis.text.y = element_text(size=8)
#     )

# options(repr.plot.width=6, repr.plot.height=7)
# plot(p)

In [ ]:
# # plotting distribution
# p = para_per_host %>%
#     filter(CI_low > 0) %>%
#     mutate(scientific_name = scientific_name %>% reorder(CI_high)) %>%
#     ggplot(aes(scientific_name, median_n_links, color=class)) +
#     geom_linerange(aes(ymin=CI_low,
#                        ymax=CI_high)) +
#     geom_point(aes(y=CI_low), size=0.5) +
#     geom_point(aes(y=CI_high), size=0.5) +
#     labs(y='No. of links (95% CI)', x='Host') +
#     coord_flip() +
#     theme_bw() +
#     theme(
#         axis.text.y = element_text(size=8)
#     )

# options(repr.plot.width=6, repr.plot.height=11)
# plot(p)

In [ ]:
# # plotting distribution
# p = para_per_host %>%
#     filter(CI_low > 0, 
#            class == 'Mammalia',
#            !is.na(CI_low)) %>%
#     mutate(scientific_name = scientific_name %>% reorder(CI_high)) %>%
#     ggplot(aes(scientific_name, median_n_links, color=order)) +
#     geom_linerange(aes(ymin=CI_low,
#                        ymax=CI_high)) +
#     geom_point(aes(y=CI_low), size=0.5) +
#     geom_point(aes(y=CI_high), size=0.5) +
#     labs(y='No. of links (95% CI)', x='Host') +
#     coord_flip() +
#     theme_bw() +
#     theme(
#         axis.text.y = element_text(size=8)
#     )

# options(repr.plot.width=6, repr.plot.height=8)
# plot(p)

# sessionInfo

In [ ]:
sessionInfo()

***

# Sandbox

In [ ]:
parafit_res = ape::parafit(parafit_l[[1]]$host.D,
             parafit_l[[1]]$para.D,
             parafit_l[[1]]$HP,
             nperm=99,
             correction="cailliez",
             test.links=TRUE)

parafit_res %>% length

In [ ]:
parafit_res %>% summary

In [ ]:
parafit_res$ParaFitGlobal

In [ ]:
parafit_res$p.global

In [ ]:
parafit_res$link.table %>% head

#### notes

* `F1` = ParaFit1.perm
* `F2` = ParaFit2.perm

From Legendre et al., 2002: 

> In summary, the ParaFitLink1 statistic gen- erally behaved better in simulations and
should be preferred to ParaFitLink2

In [ ]:
send_email('http://127.0.0.1:9705/notebooks/notebooks/16S/4_host_specific/parafit/01_IndD-sens.ipynb')